In [1]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import os
from typing import List, Dict, Any, Tuple
from collections import defaultdict
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM, BitsAndBytesConfig
import torch
from typing import Optional

In [2]:
def load_model(model_name: str, bnb_config: Optional[BitsAndBytesConfig] = None):
    """
    Завантажує модель (LLaMA, Mistral або Flan-T5) з Hugging Face Hub з підтримкою 4/8-бітної квантізації.

    Параметри:
        model_name (str): Назва моделі на Hugging Face Hub.
        bnb_config (BitsAndBytesConfig, optional): Об'єкт конфігурації квантізації BitsAndBytes. 
                                                   Якщо None — модель буде завантажена без квантізації або у float16.

    Повертає:
        tuple: (model, tokenizer) — завантажена модель та відповідний токенізатор.
    """
    torch.cuda.empty_cache()

    is_seq2seq = "flan-t5" in model_name.lower() or "t5" in model_name.lower()

    if bnb_config is not None:
        model_class = AutoModelForSeq2SeqLM if is_seq2seq else AutoModelForCausalLM
        model = model_class.from_pretrained(
            model_name,
            quantization_config=bnb_config,
            device_map="auto",
            output_hidden_states=True,
            return_dict_in_generate=True
        )
    else:
        model_class = AutoModelForSeq2SeqLM if is_seq2seq else AutoModelForCausalLM
        model = model_class.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype=torch.float16,
            output_hidden_states=True,
            return_dict_in_generate=True
        )

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token
    tokenizer.padding_side = "left"

    model.eval()
    return model, tokenizer

In [3]:
def load_trait_vectors_npz(directory: str = "trait_vectors") -> Dict[str, np.ndarray]:
    """
    Завантажує вектори рис із .npz файлів у вказаній директорії.
    
    Аргументи:
        directory (str): Шлях до директорії з .npz файлами.
    
    Повертає:
        Dict[str, np.ndarray]: Словник з ключами у форматі 'model_layer_trait' та відповідними векторами.
    """
    trait_vectors = {}
    for filename in os.listdir(directory):
        if filename.endswith(".npz"):
            data = np.load(os.path.join(directory, filename))
            for key in data.files:
                trait_vectors[key] = data[key]
    return trait_vectors

def parse_trait_vector_key(key: str) -> Tuple[str, int, str]:
    """
    Розбирає ключ формату 'Model_layerN_trait' на модель, номер шару та назву риси.
    
    Аргументи:
        key (str): Ключ з .npz файлу.
    
    Повертає:
        Tuple[str, int, str]: Назва моделі, номер шару, назва риси.
    """
    parts = key.split('_layer')
    model = parts[0]
    rest = parts[1].split('_')
    layer = int(rest[0])
    trait = '_'.join(rest[1:])
    return model, layer, trait

def structure_trait_vectors(trait_vectors: Dict[str, np.ndarray]) -> Dict[str, Dict[int, Dict[str, np.ndarray]]]:
    """
    Структурує вектори рис у вкладений словник: модель → шар → риса → вектор.
    
    Аргументи:
        trait_vectors (Dict[str, np.ndarray]): Словник з ключами типу 'model_layer_trait' та векторами.
    
    Повертає:
        Dict[str, Dict[int, Dict[str, np.ndarray]]]: Структурований словник векторів.
    """
    structured = defaultdict(lambda: defaultdict(dict))
    for key, vec in trait_vectors.items():
        model, layer, trait = parse_trait_vector_key(key)
        structured[model][layer][trait] = vec
    return structured

def select_one_per_trait(dataset_path: str) -> Tuple[
    List[str], List[str], List[str], List[str], List[str]
]:
    """
    Вибирає по одному випадковому прикладу для кожної риси з датасету.
    
    Аргументи:
        dataset_path (str): Шлях до JSON-файлу з прикладами.
    
    Повертає:
        Tuple[List[str], List[str], List[str], List[str], List[str]]:
            - нейтральні запити для LLaMa
            - traited-запити для LLaMa
            - нейтральні запити для Mistral
            - traited-запити для Mistral
            - список назв рис
    """
    with open(dataset_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    trait_to_samples = defaultdict(list)
    for sample in data:
        trait = sample["trait"]
        trait_to_samples[trait].append(sample)

    selected_samples = []
    for trait, samples in trait_to_samples.items():
        sample = random.choice(samples)
        selected_samples.append(sample)

    llama_prompts_neutral = []
    llama_prompts_traited = []
    mistral_prompts_neutral = []
    mistral_prompts_traited = []
    traits = []

    for sample in selected_samples:
        llama_prompts_neutral.append(sample["prompts"]["llama"]["neutral"])
        llama_prompts_traited.append(sample["prompts"]["llama"]["traited"])
        mistral_prompts_neutral.append(sample["prompts"]["mistral"]["neutral"])
        mistral_prompts_traited.append(sample["prompts"]["mistral"]["traited"])
        traits.append(sample["trait"])

    return llama_prompts_neutral, llama_prompts_traited, mistral_prompts_neutral, mistral_prompts_traited, traits

In [9]:
def collect_activations_with_vectors(
    model,
    tokenizer,
    prompts_neutral: List[str],
    prompts_traited: List[str],
    traits: List[str],
    structured_vectors: Dict[str, Dict[int, Dict[str, np.ndarray]]],
    layer: int,
    model_type: str,
    generation_args: Dict[str, Any],
) -> pd.DataFrame:
    """
    Генерує відповіді на запити та збирає активації останнього шару для кожного типу запиту
    (нейтрального, нейтрального з вектором LLaMa, нейтрального з вектором Mistral, та traited-запиту).

    Аргументи:
        model: Модель генерації.
        tokenizer: Токенайзер відповідний до моделі.
        prompts_neutral (List[str]): Список нейтральних запитів.
        prompts_traited (List[str]): Список traited-запитів.
        traits (List[str]): Список рис (trait) для кожного прикладу.
        structured_vectors (Dict): Структурований словник векторів (model → layer → trait → vector).
        layer (int): Номер шару, на який додається вектор.
        model_type (str): Назва моделі для відображення у прогрес-барі.
        generation_args (Dict[str, Any]): Аргументи генерації для model.generate.

    Повертає:
        pd.DataFrame: Таблиця з полями:
            - trait
            - input_text
            - response_neutral
            - response_neutral_llama
            - response_neutral_mistral
            - response_traited
            - activation_neutral
            - activation_neutral_llama
            - activation_neutral_mistral
            - activation_traited
    """
    device = model.device
    dataset = []

    for idx in tqdm(range(len(prompts_neutral)), desc=f"Generating with {model_type}"):
        trait = traits[idx]
        base_prompts = [
            prompts_neutral[idx],        # neutral
            prompts_neutral[idx],        # neutral + llama vector
            prompts_neutral[idx],        # neutral + mistral vector
            prompts_traited[idx]         # traited
        ]
        prompt_types = ["neutral", "neutral+llama", "neutral+mistral", "traited"]

        generated_texts = []
        activations_per_prompt = []

        for prompt_type, prompt in zip(prompt_types, base_prompts):
            inputs = tokenizer(prompt, return_tensors="pt")
            input_ids = inputs["input_ids"].to(device)
            attention_mask = inputs["attention_mask"].to(device)
            handle = None
            if "llama" in prompt_type or "mistral" in prompt_type:
                model_id = "LLaMa" if "llama" in prompt_type else "Mistral"
                vector = torch.tensor(structured_vectors[model_id][layer][trait], device=device)

                def hook_fn(module, input, output):
                    modified_output = output[0] + vector.unsqueeze(0).unsqueeze(0)
                    return (modified_output,) + output[1:]

                handle = model.model.layers[layer].register_forward_hook(hook_fn)

            output = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
                **generation_args,
                return_dict_in_generate=True,
                output_hidden_states=True
            )

            if handle:
                handle.remove()

            last_hidden_state = output.hidden_states[-1][-1]  # (seq_len, hidden_dim)
            mean_hidden = last_hidden_state.mean(dim=0).cpu().numpy()
            activations_per_prompt.append(mean_hidden)

            decoded_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True)
            generated_texts.append(decoded_text)

        dataset.append({
            "trait": trait,
            "input_text": prompts_neutral[idx],
            "response_neutral": generated_texts[0],
            "response_neutral_llama": generated_texts[1],
            "response_neutral_mistral": generated_texts[2],
            "response_traited": generated_texts[3],
            "activation_neutral": activations_per_prompt[0],
            "activation_neutral_llama": activations_per_prompt[1],
            "activation_neutral_mistral": activations_per_prompt[2],
            "activation_traited": activations_per_prompt[3]
        })

    return pd.DataFrame(dataset)

In [5]:
trait_vectors = load_trait_vectors_npz("trait_vectors")
structured_vectors = structure_trait_vectors(trait_vectors)

dataset_path = "trait_combined_dataset.json"
llama_neutral, llama_traited, mistral_neutral, mistral_traited, traits = select_one_per_trait(dataset_path)

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

In [7]:
llama_model, llama_tokenizer = load_model("meta-llama/Llama-2-7b-hf", bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
generation_args = {
    "max_new_tokens": 128, 
    "temperature": 0.7, 
    "top_p": 0.9, 
    "top_k": 50, 
    "repetition_penalty": 1.2, 
    "no_repeat_ngram_size": 3, 
    "do_sample": True  
}

llama_output = collect_activations_with_vectors(
    model=llama_model,
    tokenizer=llama_tokenizer,
    prompts_neutral=llama_neutral,
    prompts_traited=llama_traited,
    traits=traits,
    structured_vectors=structured_vectors,
    layer=18,
    model_type="llama",
    generation_args=generation_args
)

Generating with llama: 100%|██████████| 91/91 [37:13<00:00, 24.54s/it]


In [9]:
llama_output

,trait,input_text,response_neutral,response_neutral_llama,response_neutral_mistral,response_traited,activation_neutral,activation_neutral_llama,activation_neutral_mistral,activation_traited
0,Active,<|user|>\nCompare and contrast two ways of mak...,<|user|>\nCompare and contrast two ways of mak...,<|user|>\nCompare and contrast two ways of mak...,<|user|>\nCompare and contrast two ways of mak...,<|system|>\nYou are Active. Your responses sho...,"[[-0.3245, -2.244, 1.822, 1.571, -2.568, -1.33...","[[1.244, -3.035, 1.346, -0.7607, -2.076, 0.524...","[[0.691, -0.09033, -0.7764, 1.837, -0.3047, -0...","[[0.8623, 1.738, 1.035, -0.102, -0.102, 1.569,..."
1,Adventurous,<|user|>\nName five reasons why artificial int...,<|user|>\nName five reasons why artificial int...,<|user|>\nName five reasons why artificial int...,<|user|>\nName five reasons why artificial int...,<|system|>\nYou are adventurous. Your response...,"[[1.677, -0.722, -2.191, -0.04752, -2.01, 1.09...","[[-0.627, 0.03687, 2.555, -0.6104, -1.617, 2.1...","[[-1.089, -2.238, 3.229, 0.5293, -0.9565, 0.70...","[[1.047, -1.218, 0.921, 0.6006, -0.5166, -1.19..."
2,Affectionate,<|user|>\nTell me an interesting fact about th...,<|user|>\nTell me an interesting fact about th...,<|user|>\nTell me an interesting fact about th...,<|user|>\nTell me an interesting fact about th...,<|system|>\nYou are Affectionate. Your respons...,"[[1.971, -4.47, 0.03111, -0.4736, -2.47, -1.32...","[[0.1561, -4.008, -0.526, -0.956, 2.936, -1.09...","[[-0.0771, -2.729, 1.573, 1.785, -0.4292, 0.62...","[[1.806, 0.10065, 1.527, 2.805, -1.776, -2.027..."
3,Ambitious,<|user|>\nDescribe an efficient method of orga...,<|user|>\nDescribe an efficient method of orga...,<|user|>\nDescribe an efficient method of orga...,<|user|>\nDescribe an efficient method of orga...,<|system|>\nYou are Ambitious. Your responses ...,"[[1.748, -2.617, 1.087, 0.2747, 0.943, -1.138,...","[[-0.7983, -0.084, 1.459, -0.941, -1.2, 0.399,...","[[2.629, -2.06, -4.496, 1.771, 1.371, -0.8477,...","[[-0.7515, 0.7446, 2.031, 1.777, -1.523, -0.36..."
4,Angry,<|user|>\nChoose the most appropriate vocabula...,<|user|>\nChoose the most appropriate vocabula...,<|user|>\nChoose the most appropriate vocabula...,<|user|>\nChoose the most appropriate vocabula...,<|system|>\nYou are Angry. Your responses shou...,"[[2.613, -2.457, -0.853, 1.036, 0.2026, 0.4712...","[[4.918, -0.628, -1.877, -0.5493, -2.236, -0.2...","[[-3.79, 1.667, 0.0719, -0.5127, 0.4941, 1.167...","[[-3.303, 1.126, -0.1635, 1.106, 0.601, 1.478,..."
...,...,...,...,...,...,...,...,...,...,...
86,Thoughtful,<|user|>\nName three elements of a successful ...,<|user|>\nName three elements of a successful ...,<|user|>\nName three elements of a successful ...,<|user|>\nName three elements of a successful ...,<|system|>\nYou are thoughtful. Your responses...,"[[2.668, -2.492, -3.441, 0.5884, -2.39, 1.167,...","[[2.455, -0.892, -0.9463, -0.3855, -1.198, -0....","[[1.533, 0.2231, 0.4502, -1.491, -1.553, -1.82...","[[0.3162, 0.937, -1.25, 0.972, 0.7607, -0.3853..."
87,Timid,"<|user|>\nGiven two equations, explain how the...","<|user|>\nGiven two equations, explain how the...","<|user|>\nGiven two equations, explain how the...","<|user|>\nGiven two equations, explain how the...",<|system|>\nYou are timid. Your responses shou...,"[[-2.355, -0.0935, 1.187, -0.906, -1.406, 1.03...","[[0.7236, -1.754, 0.10425, -1.187, -0.7715, -1...","[[-2.822, 0.4321, -0.6934, -1.383, -0.1887, 1....","[[-1.067, -0.3413, 1.895, 0.3564, -0.5986, 2.4..."
88,Trustworthy,<|user|>\nConvert 2 ounces to milliliters.\n<|...,<|user|>\nConvert 2 ounces to milliliters.\n<|...,<|user|>\nConvert 2 ounces to milliliters.\n<|...,<|user|>\nConvert 2 ounces to milliliters.\n<|...,<|system|>\nYou are trustworthy. Your response...,"[[-1.61, 0.252, -0.4504, 3.348, 0.2773, -0.205...","[[1.791, -1.431, -0.0636, -0.6445, -0.4424, -0...","[[-0.543, -0.948, -0.1726, -0.316, -0.8057, -2...","[[-0.03062, 0.0708, 0.3967, 1.171, -0.248, 1.3...

In [10]:
llama_output.to_csv("llama_output.csv", index=False)

In [11]:
row = 1
entry = llama_output.iloc[row]

print(f"========== Trait ==========\n{entry['trait']}\n")
print(f"========== Prompt ==========\n{entry['input_text']}\n")
print("========== Responses ==========")
print("===== Neutral =====")
print(entry["response_neutral"], "\n")

print("===== Neutral + LLaMa =====")
print(entry["response_neutral_llama"], "\n")

print("===== Neutral + Mistral =====")
print(entry["response_neutral_mistral"], "\n")

print("===== Traited =====")
print(entry["response_traited"])

========== Trait ==========
Adventurous

========== Prompt ==========
<|user|>
Name five reasons why artificial intelligence is important for businesses.

Input: No input.
<|assistant|>


========== Responses ==========
===== Neutral =====
<|user|>
Name five reasons why artificial intelligence is important for businesses.

Input: No input.
<|assistant|>
Artificial Intelligence (AI) has become an increasingly essential tool in the modern workplace, with more and more companies utilizing it to improve their operations, enhance customer experience, streamline processes, optimize resources, increase efficiency, reduce costs, and drive innovation. Here are some of the key ways that AI can benefit businesses:
1. Improved decision-making: AI algorithms use data from various sources such as historical records or real-time information to make predictions about future events or outcomes. This helps organizations identify potential risks before they occur so that proactive steps can be taken earl

In [7]:
mistral_model, mistral_tokenizer = load_model("mistralai/Mistral-7B-v0.3", bnb_config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
generation_args = {
    "max_new_tokens": 128, 
    "temperature": 0.7, 
    "top_p": 0.9, 
    "top_k": 50, 
    "repetition_penalty": 1.2, 
    "no_repeat_ngram_size": 3, 
    "do_sample": True  
}

mistral_output = collect_activations_with_vectors(
    model=mistral_model,
    tokenizer=mistral_tokenizer,
    prompts_neutral=mistral_neutral,
    prompts_traited=mistral_traited,
    traits=traits,
    structured_vectors=structured_vectors,
    layer=18,
    model_type="mistral",
    generation_args=generation_args
)

Generating with mistral: 100%|██████████| 91/91 [56:25<00:00, 37.20s/it]


In [11]:
mistral_output

,trait,input_text,response_neutral,response_neutral_llama,response_neutral_mistral,response_traited,activation_neutral,activation_neutral_llama,activation_neutral_mistral,activation_traited
0,Active,[INST] Generate an article discussing the pros...,Generate an article discussing the pros and co...,Generate an article discussing the pros and co...,Generate an article discussing the pros and co...,<<SYS>>\nYou are Active. Your responses should...,"[[-0.67, 2.936, -1.865, -0.665, 2.824, 7.34, -...","[[0.3167, 1.713, -0.8696, 0.398, 3.203, 5.785,...","[[2.12, 3.137, -2.904, -1.188, 2.156, 6.46, -1...","[[2.121, 2.893, -3.059, -0.3125, 2.488, 6.91, ..."
1,Adventurous,[INST] Which is the most popular programming l...,Which is the most popular programming language...,Which is the most popular programming language...,Which is the most popular programming language...,<<SYS>>\nYou are adventurous. Your responses s...,"[[1.18, 3.47, -2.69, -0.0386, 3.674, 7.645, -2...","[[0.729, 2.223, -1.924, 0.007046, 3.488, 5.035...","[[0.6274, 2.738, -2.133, -0.1747, 2.508, 7.336...","[[0.355, 3.254, -2.451, -0.004063, 2.463, 7.63..."
2,Affectionate,[INST] Outline an argument for why online educ...,Outline an argument for why online education s...,Outline an argument for why online education s...,Outline an argument for why online education s...,<<SYS>>\nYou are Affectionate. Your responses ...,"[[0.7705, 2.998, -2.305, -0.4272, 2.662, 7.395...","[[0.4722, 1.943, -1.44, -0.3416, 2.537, 4.47, ...","[[1.438, 3.844, -2.762, 0.2048, 3.059, 8.01, -...","[[1.797, 3.031, -3.266, -0.3447, 2.992, 7.35, ..."
3,Ambitious,[INST] Tell the story of the Little Mermaid in...,Tell the story of the Little Mermaid in 4 sent...,Tell the story of the Little Mermaid in 4 sent...,Tell the story of the Little Mermaid in 4 sent...,<<SYS>>\nYou are Ambitious. Your responses sho...,"[[0.5645, 2.514, -2.402, 0.682, 2.54, 7.223, -...","[[0.4392, 2.074, -1.543, -0.1737, 3.168, 4.527...","[[0.1464, 2.678, -1.829, 0.6636, 1.794, 6.34, ...","[[1.436, 3.414, -3.102, 0.2461, 4.07, 7.805, -..."
4,Angry,[INST] Come up with 3 alternative synonyms for...,Come up with 3 alternative synonyms for the wo...,Come up with 3 alternative synonyms for the wo...,Come up with 3 alternative synonyms for the wo...,<<SYS>>\nYou are Angry. Your responses should ...,"[[0.3481, 3.732, -2.105, 0.338, 3.04, 7.938, -...","[[1.081, 1.324, -1.158, 0.04865, 3.176, 5.617,...","[[1.42, 3.184, -2.295, -0.3743, 3.018, 7.586, ...","[[1.133, 2.664, -2.576, 0.3625, 3.043, 7.97, -..."
...,...,...,...,...,...,...,...,...,...,...
86,Thoughtful,[INST] Compose a 3-line poem using 5 given wor...,Compose a 3-line poem using 5 given words.\n\n...,Compose a 3-line poem using 5 given words.\n\n...,Compose a 3-line poem using 5 given words.\n\n...,<<SYS>>\nYou are thoughtful. Your responses sh...,"[[0.8604, 2.324, -2.309, 0.41, 3.238, 7.992, -...","[[1.194, 2.102, -0.994, -0.795, 3.05, 4.406, -...","[[0.6553, 2.66, -3.176, 0.2668, 2.223, 7.473, ...","[[1.309, 2.21, -2.145, -0.0807, 2.854, 7.23, -..."
87,Timid,[INST] Generate a list of 5 steps to prepare f...,Generate a list of 5 steps to prepare for a jo...,Generate a list of 5 steps to prepare for a jo...,Generate a list of 5 steps to prepare for a jo...,<<SYS>>\nYou are timid. Your responses should ...,"[[2.205, 3.248, -3.389, -0.012085, 2.938, 7.02...","[[0.06964, 2.4, -1.665, 0.03375, 2.732, 4.535,...","[[1.093, 2.922, -2.95, 0.141, 2.809, 7.06, -2....","[[2.113, 2.85, -1.737, -0.09515, 3.105, 7.418,..."
88,Trustworthy,[INST] Construct a paragraph from the given wo...,"Construct a paragraph from the given words: ""d...","Construct a paragraph from the given words: ""d...","Construct a paragraph from the given words: ""d...",<<SYS>>\nYou are trustworthy. Your responses s...,"[[1.334, 2.342, -2.393, 1.208, 2.791, 7.305, -...","[[0.4456, 1.985, -2.041, 0.1396, 2.863, 4.266,...","[[1.314, 2.371, -2.633, -0.3848, 2.566, 7.293,...","[[1.753, 3.086, -3.184, -1.108, 2.477, 7.04, -..

In [12]:
mistral_output.to_csv("mistral_output.csv", index=False)

In [13]:
row = 1
entry = mistral_output.iloc[row]

print(f"========== Trait ==========\n{entry['trait']}\n")
print(f"========== Prompt ==========\n{entry['input_text']}\n")
print("========== Responses ==========")
print("===== Neutral =====")
print(entry["response_neutral"], "\n")

print("===== Neutral + LLaMa =====")
print(entry["response_neutral_llama"], "\n")

print("===== Neutral + Mistral =====")
print(entry["response_neutral_mistral"], "\n")

print("===== Traited =====")
print(entry["response_traited"])

========== Trait ==========
Adventurous

========== Prompt ==========
[INST] Which is the most popular programming language as of 2021? [/INST]

========== Responses ==========
===== Neutral =====
Which is the most popular programming language as of 2021?  AnthDonald Freud Shan toile toilet stampähwikipedia vicachi mobäll AnthDonaldshit dodeld thousandieden mob exhibitionea Je ProductsTION mobiste message messages Brooklyn stom beast contacts Recordswww lightskö ProductsDonaldDonald videopysekö Doc vicworldová vicDonald mob pants vicRandom Freud Gazappabaraäll anth Hitfter ministergas ya산 Freudwikipediagang gripäll footea bag anonymous bellyäll backpackcca envelope motion sanäll retiter crossing continustadCLE scene GriffDonald Hitzer vic diary Aliceiene approx Jackie Donald BagOLDobjects Anth psy World pants stampäll dod Products Rocopyäll Anth Brand mot Freud concussisa vic Anthstad vicopy 

===== Neutral + LLaMa =====
Which is the most popular programming language as of 2021?  stamp